In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2

# below imports are used to print out pretty pandas dataframes
from IPython.display import display, HTML

%matplotlib inline
plt.style.use('ggplot')

# information used to create a database connection
sqluser = 'postgres'
dbname = 'mimic'
schema_name = 'mimiciii'
pw = 'postgres'

# Connect to postgres with a copy of the MIMIC-III database
con = psycopg2.connect(dbname=dbname, user=sqluser, password=pw)

# the below statement is prepended to queries to ensure they select from the right schema
query_schema = 'set search_path to ' + schema_name + ';'

In [2]:
#get data from table: patients
query = query_schema +"""
SELECT subject_id, gender, dob, dod, expire_flag
FROM patients
"""
patients = pd.read_sql_query(query, con)

In [3]:
#get data from table: prescriptions
query = query_schema +"""
SELECT row_id, subject_id, hadm_id, icustay_id, startdate, enddate, drug_type, drug, drug_name_poe, drug_name_generic, formulary_drug_cd
FROM prescriptions
"""
prescriptions = pd.read_sql_query(query, con)

In [4]:
#get data from table: diagnoses_icd
query = query_schema +"""
SELECT subject_id, hadm_id, icd9_code
FROM diagnoses_icd
"""
diagnoses_icd = pd.read_sql_query(query, con)

In [5]:
#get data from table: d_diagnoses_icd
query = query_schema +"""
SELECT short_title, icd9_code, long_title
FROM d_icd_diagnoses
"""
d_diagnoses_icd = pd.read_sql_query(query, con)

In [6]:
#get data from table: admissions
query = query_schema +"""
SELECT subject_id, hadm_id, diagnosis, dischtime, deathtime, admittime
FROM admissions
"""
admissions = pd.read_sql_query(query, con)

In [7]:
#get data from table: drgcodes
query = query_schema +"""
SELECT subject_id, hadm_id, drg_code, drg_severity, drg_mortality
FROM drgcodes
"""
drgcodes = pd.read_sql_query(query, con)

In [8]:
admissions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58976 entries, 0 to 58975
Data columns (total 6 columns):
subject_id    58976 non-null int64
hadm_id       58976 non-null int64
diagnosis     58951 non-null object
dischtime     58976 non-null datetime64[ns]
deathtime     5854 non-null datetime64[ns]
admittime     58976 non-null datetime64[ns]
dtypes: datetime64[ns](3), int64(2), object(1)
memory usage: 2.7+ MB


#### Reprocessing dates from random future dates to MIMIC dataframe date range

In [9]:
#make a copy of admissions
admissions2=admissions.copy()

In [10]:
#group future dates into 2001-2012
admissions2['admit_year'] = pd.Series(np.zeros(admissions2.shape[0]))
admissions2.loc[admissions2['admittime'].dt.year>2201, 'admit_year'] = int(2012)
admissions2.loc[(admissions2['admittime'].dt.year<=2201) & (admissions2['admittime'].dt.year>2192), 'admit_year'] = int(2011)
admissions2.loc[(admissions2['admittime'].dt.year<=2192) & (admissions2['admittime'].dt.year>2183), 'admit_year'] = 2010
admissions2.loc[(admissions2['admittime'].dt.year<=2183) & (admissions2['admittime'].dt.year>2174), 'admit_year'] = 2009
admissions2.loc[(admissions2['admittime'].dt.year<=2174) & (admissions2['admittime'].dt.year>2165), 'admit_year'] = 2008
admissions2.loc[(admissions2['admittime'].dt.year<=2165) & (admissions2['admittime'].dt.year>2156), 'admit_year'] = 2007
admissions2.loc[(admissions2['admittime'].dt.year<=2156) & (admissions2['admittime'].dt.year>2147), 'admit_year'] = 2006
admissions2.loc[(admissions2['admittime'].dt.year<=2147) & (admissions2['admittime'].dt.year>2138), 'admit_year'] = 2005
admissions2.loc[(admissions2['admittime'].dt.year<=2138) & (admissions2['admittime'].dt.year>2129), 'admit_year'] = 2004
admissions2.loc[(admissions2['admittime'].dt.year<=2129) & (admissions2['admittime'].dt.year>2120), 'admit_year'] = 2003
admissions2.loc[(admissions2['admittime'].dt.year<=2120) & (admissions2['admittime'].dt.year>2111), 'admit_year'] = 2002
admissions2.loc[admissions2['admittime'].dt.year<=2111, 'admit_year'] = 2001

In [11]:
#convert new variable to an integer
admissions2['admit_year'] = admissions2['admit_year'].astype('int')
admissions2.head()

,subject_id,hadm_id,diagnosis,dischtime,deathtime,admittime,admit_year
0,22,165315,BENZODIAZEPINE OVERDOSE,2196-04-10 15:54:00,NaT,2196-04-09 12:26:00,2011
1,23,152223,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,2153-09-08 19:10:00,NaT,2153-09-03 07:15:00,2006
2,23,124321,BRAIN MASS,2157-10-25 14:00:00,NaT,2157-10-18 19:34:00,2007
3,24,161859,INTERIOR MYOCARDIAL INFARCTION,2139-06-09 12:48:00,NaT,2139-06-06 16:14:00,2005
4,25,129635,ACUTE CORONARY SYNDROME,2160-11-05 14:55:00,NaT,2160-11-02 02:06:00,2007


In [12]:
#extract month and date
admissions2['admit_month']=admissions2['admittime'].dt.month
admissions2['admit_day']=admissions2['admittime'].dt.day

#convert all leap year days to 2004 to avoid conflict
admissions2.loc[(admissions2['admit_day']==29) & (admissions2['admit_month']==2), 'admit_year'] = 2004

In [13]:
#convert to strings
admissions2['admit_year'] = admissions2['admit_year'].astype('str')
admissions2['admit_month'] = admissions2['admit_month'].astype('str')
admissions2['admit_day'] = admissions2['admit_day'].astype('str')

#merge new variables
admissions2['admit_new']=admissions2[['admit_year', 'admit_month', 'admit_day']].apply(lambda x: '-'.join(x), axis=1)

#convert string to date
admissions2['admit_new'] = pd.to_datetime(admissions2['admit_new'])

#print dataset head
admissions2.head()

,subject_id,hadm_id,diagnosis,dischtime,deathtime,admittime,admit_year,admit_month,admit_day,admit_new
0,22,165315,BENZODIAZEPINE OVERDOSE,2196-04-10 15:54:00,NaT,2196-04-09 12:26:00,2011,4,9,2011-04-09
1,23,152223,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,2153-09-08 19:10:00,NaT,2153-09-03 07:15:00,2006,9,3,2006-09-03
2,23,124321,BRAIN MASS,2157-10-25 14:00:00,NaT,2157-10-18 19:34:00,2007,10,18,2007-10-18
3,24,161859,INTERIOR MYOCARDIAL INFARCTION,2139-06-09 12:48:00,NaT,2139-06-06 16:14:00,2005,6,6,2005-06-06
4,25,129635,ACUTE CORONARY SYNDROME,2160-11-05 14:55:00,NaT,2160-11-02 02:06:00,2007,11,2,2007-11-02


In [14]:
#remove all new variables except converted date
cols=[6,7,8]
admissions2.drop(admissions2.columns[cols], axis=1,inplace=True)

#print dataframe info
admissions2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58976 entries, 0 to 58975
Data columns (total 7 columns):
subject_id    58976 non-null int64
hadm_id       58976 non-null int64
diagnosis     58951 non-null object
dischtime     58976 non-null datetime64[ns]
deathtime     5854 non-null datetime64[ns]
admittime     58976 non-null datetime64[ns]
admit_new     58976 non-null datetime64[ns]
dtypes: datetime64[ns](4), int64(2), object(1)
memory usage: 3.1+ MB


In [15]:
#repeating above process for discharge date
admissions2['disch_year'] = pd.Series(np.zeros(admissions2.shape[0]))
admissions2.loc[admissions2['dischtime'].dt.year>2201, 'disch_year'] = 2012
admissions2.loc[(admissions2['dischtime'].dt.year<=2201) & (admissions2['dischtime'].dt.year>2192), 'disch_year'] = 2011
admissions2.loc[(admissions2['dischtime'].dt.year<=2192) & (admissions2['dischtime'].dt.year>2183), 'disch_year'] = 2010
admissions2.loc[(admissions2['dischtime'].dt.year<=2183) & (admissions2['dischtime'].dt.year>2174), 'disch_year'] = 2009
admissions2.loc[(admissions2['dischtime'].dt.year<=2174) & (admissions2['dischtime'].dt.year>2165), 'disch_year'] = 2008
admissions2.loc[(admissions2['dischtime'].dt.year<=2165) & (admissions2['dischtime'].dt.year>2156), 'disch_year'] = 2007
admissions2.loc[(admissions2['dischtime'].dt.year<=2156) & (admissions2['dischtime'].dt.year>2147), 'disch_year'] = 2006
admissions2.loc[(admissions2['dischtime'].dt.year<=2147) & (admissions2['dischtime'].dt.year>2138), 'disch_year'] = 2005
admissions2.loc[(admissions2['dischtime'].dt.year<=2138) & (admissions2['dischtime'].dt.year>2129), 'disch_year'] = 2004
admissions2.loc[(admissions2['dischtime'].dt.year<=2129) & (admissions2['dischtime'].dt.year>2120), 'disch_year'] = 2003
admissions2.loc[(admissions2['dischtime'].dt.year<=2120) & (admissions2['dischtime'].dt.year>2111), 'disch_year'] = 2002
admissions2.loc[admissions2['dischtime'].dt.year<=2111, 'disch_year'] = 2001

In [16]:
admissions2['disch_year'] = admissions2['disch_year'].astype('int')
admissions2.head()

,subject_id,hadm_id,diagnosis,dischtime,deathtime,admittime,admit_new,disch_year
0,22,165315,BENZODIAZEPINE OVERDOSE,2196-04-10 15:54:00,NaT,2196-04-09 12:26:00,2011-04-09,2011
1,23,152223,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,2153-09-08 19:10:00,NaT,2153-09-03 07:15:00,2006-09-03,2006
2,23,124321,BRAIN MASS,2157-10-25 14:00:00,NaT,2157-10-18 19:34:00,2007-10-18,2007
3,24,161859,INTERIOR MYOCARDIAL INFARCTION,2139-06-09 12:48:00,NaT,2139-06-06 16:14:00,2005-06-06,2005
4,25,129635,ACUTE CORONARY SYNDROME,2160-11-05 14:55:00,NaT,2160-11-02 02:06:00,2007-11-02,2007


In [17]:
admissions2['disch_month']=admissions2['dischtime'].dt.month
admissions2['disch_day']=admissions2['dischtime'].dt.day

admissions2.loc[(admissions2['disch_day']==29) & (admissions2['disch_month']==2), 'disch_year'] = 2004

In [18]:
admissions2['disch_year'] = admissions2['disch_year'].astype('str')
admissions2['disch_month'] = admissions2['disch_month'].astype('str')
admissions2['disch_day'] = admissions2['disch_day'].astype('str')

admissions2['disch_new']=admissions2[['disch_year', 'disch_month', 'disch_day']].apply(lambda x: '-'.join(x), axis=1)

admissions2['disch_new'] = pd.to_datetime(admissions2['disch_new'])

admissions2.head()

,subject_id,hadm_id,diagnosis,dischtime,deathtime,admittime,admit_new,disch_year,disch_month,disch_day,disch_new
0,22,165315,BENZODIAZEPINE OVERDOSE,2196-04-10 15:54:00,NaT,2196-04-09 12:26:00,2011-04-09,2011,4,10,2011-04-10
1,23,152223,CORONARY ARTERY DISEASE\CORONARY ARTERY BYPASS...,2153-09-08 19:10:00,NaT,2153-09-03 07:15:00,2006-09-03,2006,9,8,2006-09-08
2,23,124321,BRAIN MASS,2157-10-25 14:00:00,NaT,2157-10-18 19:34:00,2007-10-18,2007,10,25,2007-10-25
3,24,161859,INTERIOR MYOCARDIAL INFARCTION,2139-06-09 12:48:00,NaT,2139-06-06 16:14:00,2005-06-06,2005,6,9,2005-06-09
4,25,129635,ACUTE CORONARY SYNDROME,2160-11-05 14:55:00,NaT,2160-11-02 02:06:00,2007-11-02,2007,11,5,2007-11-05


In [19]:
cols=[7,8,9]
admissions2.drop(admissions2.columns[cols], axis=1,inplace=True)

admissions2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58976 entries, 0 to 58975
Data columns (total 8 columns):
subject_id    58976 non-null int64
hadm_id       58976 non-null int64
diagnosis     58951 non-null object
dischtime     58976 non-null datetime64[ns]
deathtime     5854 non-null datetime64[ns]
admittime     58976 non-null datetime64[ns]
admit_new     58976 non-null datetime64[ns]
disch_new     58976 non-null datetime64[ns]
dtypes: datetime64[ns](5), int64(2), object(1)
memory usage: 3.6+ MB


### Merge imported datasets

In [20]:
df_merge1 = pd.merge(patients, diagnoses_icd, how='inner', left_on='subject_id', right_on='subject_id')

In [21]:
df_merge2 = pd.merge(df_merge1, d_diagnoses_icd, how='inner', left_on='icd9_code', right_on='icd9_code')

In [22]:
df_merge3 = pd.merge(df_merge2, admissions2, how='inner', left_on=['hadm_id', 'subject_id'], right_on=['hadm_id', 'subject_id'])

In [23]:
df_merge4 = pd.merge(df_merge3, drgcodes, how='inner', left_on=['hadm_id', 'subject_id'], right_on=['hadm_id', 'subject_id'])

In [24]:
#trying to save memory
del([diagnoses_icd, d_diagnoses_icd, admissions, admissions2, drgcodes, df_merge1, df_merge2, df_merge3]) 

In [25]:
#export datasets
df_merge4.to_csv("C:/Users/Maggie/OneDrive/UW-BHI/2018Fall/CSE583/Project/mimic_merge4.csv")
prescriptions.to_csv("C:/Users/Maggie/OneDrive/UW-BHI/2018Fall/CSE583/Project/mimic_prescriptions.csv")